In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [2]:
gmsh.open("test-twist-3D.geo")

Info    : Reading 'test-twist-3D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 40%] Meshing curve 2 (Line)
Info    : [ 70%] Meshing curve 3 (Circle)
Info    : Done meshing 1D (Wall 0.00159283s, CPU 0.001592s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Cylinder, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0782686s, CPU 0.077972s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 351 nodes...
Info    : Done tetrahedrizing 359 nodes (Wall 0.0145917s, CPU 0.013618s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.0320573s, CPU 0.029034s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 2.0

In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat])

Problem("test-twist-3D", :Solid, 3, 3, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 2648, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [4]:
supp = FEM.displacementConstraint("supp", ux=0, uy=0)
ff(x, y, z) = √(y^2 + z^2)
load = FEM.load("load", fy=ff)

BoundaryCondition("load", nothing, nothing, nothing, nothing, nothing, nothing, nothing, Main.ff, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [5]:
n1x(x, y, z) = y
n1y(x, y, z) = z
n2x(x, y, z) = -z
n2y = n1x
cs = FEM.CoordinateSystem([n1x, n1y, 0], [n2x, n2y, 0])
Q = FEM.rotateNodes(problem, "body", cs)

LowLevelFEM.Transformation(sparse([1, 2, 1, 2, 3, 4, 5, 4, 5, 6  …  7939, 7940, 7939, 7940, 7941, 7942, 7943, 7942, 7943, 7944], [1, 1, 2, 2, 3, 4, 4, 5, 5, 6  …  7939, 7939, 7940, 7940, 7941, 7942, 7942, 7943, 7943, 7944], [2.4492935982947065e-17, 1.0, -1.0, 2.4492935982947065e-17, 1.0, 0.9107991560957914, 0.41284972720736285, -0.4128497272073629, 0.9107991560957914, 0.9999999999999998  …  0.1687220986435244, 0.9856636614126164, -0.9856636614126164, 0.16872209864352436, 1.0, 0.015058119843009733, 0.9998866200858942, -0.9998866200858942, 0.015058119843009735, 0.9999999999999999], 7944, 7944), 2648, 3)

In [6]:
K = FEM.stiffnessMatrix(problem)
f = FEM.loadVector(problem, [load])

VectorField
[0.0; 4.604702850611635; … ; 0.0; 0.0;;]

In [7]:
K = Q' * K * Q

sparse([1, 2, 3, 7, 8, 9, 40, 41, 42, 43  …  6831, 6910, 6911, 6912, 7333, 7334, 7335, 7942, 7943, 7944], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  7944, 7944, 7944, 7944, 7944, 7944, 7944, 7944, 7944, 7944], [334508.8803204338, 30495.72163303013, 13730.56305560819, 11815.130165807039, 2542.064105000919, -3657.998686047544, 23452.269636917496, -3111.4588550931394, 7115.3748457787315, -47320.654072607555  …  59252.775514957684, 17273.7680131243, 28130.102443006108, -204640.02224287562, 277799.1848642338, 333489.28243336274, -393458.52857509465, 191695.10761677104, -569603.1351208343, 2.835769743916942e6], 7944, 7944)

In [10]:
q = FEM.solveDisplacement(K, f, support=[supp])

VectorField
[2.8940408684822586e13; 3.3627956725479117e13; … ; 2.987290471952583e13; 5.007332643096688e12;;]

In [11]:
q1 = Q * q

VectorField
[-3.3627956725479117e13; 2.8940408684822586e13; … ; 2.5382730294706156e13; 5.0073326430966875e12;;]

In [12]:
FEM.probe(q, 10, 1, 0)

3-element Vector{Float64}:
 0.0
 0.0
 9.388135250356467e12

In [13]:
FEM.showDoFResults(q, :uvec, visible=true)
FEM.showDoFResults(q1, :uvec, visible=true)

2

In [14]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [15]:
gmsh.finalize()